# Travail final: Analyse d'un sous corpus portant sur les jeux olympique de 1924 

Dans ce notebook, nous étudions de manière transversale l'enssemble du sous corpus Camile extrait.

## Imports

In [1]:
# === IMPORTS ===

import os
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Traitement du langage
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
from collections import Counter

# NLP avancé
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

# Word embeddings
from gensim.models import Word2Vec

# Nettoyage
import re
import string

# Affichage
sns.set(style="whitegrid")


## Chargement et exploration du sous corpus

In [2]:
# --- Chargement du corpus CAMille (Jeux Olympiques 1924) ---

import pandas as pd
from pathlib import Path

# Chemin du fichier (à adapter si différent)
data_path = Path("data/camille_Jeux_olympiques.xlsx")

# Chargement du fichier Excel
df = pd.read_excel(data_path)

# Aperçu des premières lignes
df.head()


,ID,JOURNAL,DATE,ANNÉE,MOIS,JOUR,JDLS,ÉDITION,PAGE,LANGUE,TEXTE
0,KB_JB838_1924-07-11_01-00005,JB838,1924-07-11,1924,7,11,5,1,5,fr-BE,"Seul 1e Marathon des Flandres n’aura pas lieu,..."
1,KB_JB838_1924-06-30_01-00001,JB838,1924-06-30,1924,6,30,1,1,1,fr-BE,"Paris est en pleines Olympiades, ce qui signif..."
2,KB_JB838_1924-04-26_01-00005,JB838,1924-04-26,1924,4,26,6,1,5,fr-BE,LA PORTÉE DES <kw>JEUX</kw> <kw>OLYMPIQUES</kw...
3,KB_JB838_1924-08-07_01-00007,JB838,1924-08-07,1924,8,7,4,1,7,fr-BE,Après lès Jeta Oiynpiqaéi UN PCU DS «TATISTIQU...
4,KB_JB838_1924-08-07_01-00005,JB838,1924-08-07,1924,8,7,4,1,5,fr-BE,Pour vos disponibilités un placement rémunérat...


In [3]:
# Ajouter une colonne 'nb_mots' basée sur le texte
df["nb_mots"] = df["TEXTE"].str.split().str.len()

# Longueur moyenne
longueur_moy = df["nb_mots"].mean()

print(f"Longueur moyenne des articles : {longueur_moy:.2f} mots")


Longueur moyenne des articles : 507.15 mots


In [4]:
# Affichage de quelques texte bruts
# Exemple de 3 textes bruts
for i, txt in enumerate(df["TEXTE"].head(3)):
    print(f"\n--- TEXTE {i+1} ---\n")
    print(txt)



--- TEXTE 1 ---

Seul 1e Marathon des Flandres n’aura pas lieu, à. cause dos <kw>Jeux</kw> <kw>Olympiques</kw>. Celui-ci sera disputé fin du mois d'août. Le programme de la fête athlétique comportera le3 épreuves suivantes: ; , 1. Critérium international de vitesse: a) 100 m. plat; b) 150 m. plat; c) 200 m. plat, par addition de points'obtenus dans chacune de ce3 épreuves. La participation do Cockuyt et JBrochart est déjà certaine.--Des’pourparlers sont engagés aveo deux -excellents sprintera étrangers. 2. Challenge du Kilomètre. ;— Classement par trois hommes dû mémo cercle. 3. Challenge des relais. — a) Bêlais suédois (400, : 300, 200 ot 100 ui.);'. b) Bêlais olympique (1,600, 600, 400. 200 m.); c) 800 m. relaie (4xzoo m.). I 4. 5,000 mètres scratch, classement spécial par > trois hommes, pour clubs de déuxième catégorie, ~ -jdoté d'un eliallcngc. 5. 200 mètres, scratch, pour deuxième catégorie. 6 .Challenge des concours: a) Lancement du disque; : b) Lancement du javelot; 'c) Lancem

# Nétoyage et préparation du texte

In [5]:
df.columns


Index(['ID', 'JOURNAL', 'DATE', 'ANNÉE', 'MOIS', 'JOUR', 'JDLS', 'ÉDITION',
       'PAGE', 'LANGUE', 'TEXTE', 'nb_mots'],
      dtype='object')

In [6]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Stopwords FR
stopwords_fr = set(stopwords.words("french"))

def clean_text(text):
    if pd.isna(text):
        return ""
    
    # Minuscule
    text = text.lower()
    
    # Suppression ponctuation + chiffres
    text = re.sub(r"[^a-zàâçéèêëîïôûùüÿñæœ\s-]", " ", text)
    
    # Supprimer les tirets isolés
    text = re.sub(r"\s-\s", " ", text)
    
    # Tokenisation
    tokens = word_tokenize(text, language="french")
    
    # Suppression stopwords
    tokens = [t for t in tokens if t not in stopwords_fr and len(t) > 2]
    
    return " ".join(tokens)


In [7]:
# Application du nettoyage a notre dataframe
df["clean_text"] = df["TEXTE"].apply(clean_text)
df[["TEXTE", "clean_text"]].head()


,TEXTE,clean_text
0,"Seul 1e Marathon des Flandres n’aura pas lieu,...",seul marathon flandres lieu cause dos jeux oly...
1,"Paris est en pleines Olympiades, ce qui signif...",paris pleines olympiades signifie jeux olympiq...
2,LA PORTÉE DES <kw>JEUX</kw> <kw>OLYMPIQUES</kw...,portée jeux olympiques modernes natation recor...
3,Après lès Jeta Oiynpiqaéi UN PCU DS «TATISTIQU...,après lès jeta oiynpiqaéi pcu tatistiquc sport...
4,Pour vos disponibilités un placement rémunérat...,disponibilités placement rémunérateur constate...


## Analyse de fréquence

In [8]:
from collections import Counter

# Fusionner tout le texte nettoyé
all_tokens = " ".join(df["clean_text"]).split()

freq_clean = Counter(all_tokens).most_common(30)

freq_clean


[('jeux', 4353),
 ('lutte', 1182),
 ('rue', 1145),
 ('plus', 849),
 ('bruxelles', 795),
 ('mètres', 733),
 ('première', 706),
 ('deux', 681),
 ('olympiques', 649),
 ('bat', 645),
 ('prix', 639),
 ('entre', 625),
 ('deuxième', 593),
 ('cette', 588),
 ('heures', 582),
 ('van', 581),
 ('belgique', 566),
 ('quinzes', 565),
 ('victoire', 547),
 ('jeu', 541),
 ('concours', 540),
 ('tout', 520),
 ('paris', 485),
 ('tous', 483),
 ('très', 483),
 ('après', 482),
 ('bien', 450),
 ('finale', 447),
 ('grand', 438),
 ('contre', 436)]

## Bigrammes

In [10]:
print(df.columns)


Index(['ID', 'JOURNAL', 'DATE', 'ANNÉE', 'MOIS', 'JOUR', 'JDLS', 'ÉDITION',
       'PAGE', 'LANGUE', 'TEXTE', 'nb_mots', 'clean_text'],
      dtype='object')


In [12]:
import itertools
from nltk import bigrams

# Tokenisation de clean_text
df["tokens"] = df["clean_text"].str.split()

# Liste de tokens
all_tokens = list(itertools.chain.from_iterable(df["tokens"]))

# Bigrams
all_bigrams = list(bigrams(all_tokens))

print(all_bigrams[:20])


[('seul', 'marathon'), ('marathon', 'flandres'), ('flandres', 'lieu'), ('lieu', 'cause'), ('cause', 'dos'), ('dos', 'jeux'), ('jeux', 'olympiques'), ('olympiques', 'celui-ci'), ('celui-ci', 'disputé'), ('disputé', 'fin'), ('fin', 'mois'), ('mois', 'août'), ('août', 'programme'), ('programme', 'fête'), ('fête', 'athlétique'), ('athlétique', 'comportera'), ('comportera', 'épreuves'), ('épreuves', 'suivantes'), ('suivantes', 'critérium'), ('critérium', 'international')]


## Extraction de mots clés

In [13]:
import yake

# 1. Instancier l'extracteur de mots-clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)

# 2. Appliquer YAKE article par article
def extract_bigrams(text):
    """Retourne les mots-clés à deux mots (bigrammes) extraits par YAKE"""
    if not isinstance(text, str):
        return []
    keywords = kw_extractor.extract_keywords(text)
    bigrams = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:   # garder uniquement les expressions à deux mots
            bigrams.append(kw)
    return bigrams

# 3. Nouvelle colonne « keywords_bigrammes »
df["keywords_bigrams"] = df["TEXTE"].apply(extract_bigrams)

# 4. Aperçu
df[["TEXTE", "keywords_bigrams"]].head()


,TEXTE,keywords_bigrams
0,"Seul 1e Marathon des Flandres n’aura pas lieu,...","[Flandres n’aura, cause dos, Olympiades fut, B..."
1,"Paris est en pleines Olympiades, ce qui signif...","[pleines Olympiades, monde grec, caractère int..."
2,LA PORTÉE DES <kw>JEUX</kw> <kw>OLYMPIQUES</kw...,"[NATATION Records, MODERNES NATATION, battus p..."
3,Après lès Jeta Oiynpiqaéi UN PCU DS «TATISTIQU...,"[Sporting Ohrqnicle, Jeta Oiynpiqaéi, aurait a..."
4,Pour vos disponibilités un placement rémunérat...,"[Sporting Ohrqnicle, PHILANTHROPIE COLOMBOPHIL..."


Nombre de fichiers détectés : 965


['KB_JB838_1924-01-01_01-00002.txt',
 'KB_JB838_1924-01-01_01-00007.txt',
 'KB_JB838_1924-01-04_01-00005.txt',
 'KB_JB838_1924-01-05_01-00005.txt',
 'KB_JB838_1924-01-05_01-00009.txt',
 'KB_JB838_1924-01-06_01-00005.txt',
 'KB_JB838_1924-01-07_01-00004.txt',
 'KB_JB838_1924-01-08_01-00001.txt',
 'KB_JB838_1924-01-09_01-00003.txt',
 'KB_JB838_1924-01-09_01-00005.txt']

KB_JB838_1924-01-01_01-00002.txt mentions these keywords: Anton Bach, Georges Bach, If'il HX-X, COMMISSION D’ENQUÊTE, université américaine...
KB_JB838_1924-01-01_01-00007.txt mentions these keywords: Mlle Sidonie, petite ligne, Bijoux Confections, ETABLISSEMENTS EXCELSIOR, rue longue...
KB_JB838_1924-01-04_01-00005.txt mentions these keywords: VMY iJAMVW, Jeux Olympiques, Tom Pouce...
KB_JB838_1924-01-05_01-00005.txt mentions these keywords: ...
KB_JB838_1924-01-05_01-00009.txt mentions these keywords: bon état, RUE NEUVE, Monsieur Collivet...
KB_JB838_1924-01-06_01-00005.txt mentions these keywords: Dftriogr Ohjb, ’LÀNTERNE SPORtlVE, dela ’LÀNTERNE, Demain dimanche, Lêuteroe Sportive, seconde catégorie, nteé IûJmjpiI'MU...
KB_JB838_1924-01-07_01-00004.txt mentions these keywords: Wavre Sport, Racing Club...
KB_JB838_1924-01-08_01-00001.txt mentions these keywords: écoles libres...
KB_JB838_1924-01-09_01-00003.txt mentions these keywords: LUEURS D’ESPOIR, SPÈCIAUX PARIS, D’ESPOIR Pari

[('de', 15308),
 ('-', 13120),
 ('kw', 9972),
 ('a', 9197),
 ('et', 9094),
 ('la', 8946),
 ('l', 6666),
 ('le', 6332),
 ('les', 5568),
 ('d', 4683),
 ('à', 4363),
 ('jeux', 4353),
 ('du', 3940),
 ('par', 3915),
 ('s', 3776),
 ('des', 3757),
 ('en', 3637),
 ('pour', 3369),
 ('i', 2974),
 ('un', 2629)]

[('sunlight', 'savon'),
 ('côtières', 'indiquées'),
 ('rex', 'ingram'),
 ('détaillés', 'complets'),
 ('malles', 'congolaises'),
 ('aldo', 'nadi'),
 ('congolaises', 'paquebots'),
 ('stations', 'côtières'),
 ('mentionnés', 'cidessous'),
 ('montes', 'probables'),
 ('carabiniers', 'reunis'),
 ('arne', 'borg'),
 ('lundis', 'mercredis'),
 ('mercredis', 'vendredis'),
 ('centimes', 'additionnels'),
 ('pages', 'illustrations'),
 ('new', 'york'),
 ('ingram', 'tiré'),
 ('composant', 'principalement'),
 ('illustrations', 'portraits')]

## Récupération de tous les articles de la page d'accueil



## Création d'un dataframe avec les liens et les titres des articles


## Téléchargement d'un article et affichage du texte

### Nettoyage du texte à l'aide d'expressions régulières

### Création d'un fichier avec le contenu de l'article


## Pour en savoir plus